In [1]:
import os
import pathlib


import matplotlib.pyplot as plt
import pandas as pd 
import plotly.express as px
import seaborn as sns

pd.options.plotting.backend = "plotly"

import warnings
warnings.filterwarnings("ignore")

In [2]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

Setting the date to April 1st 2022

In [3]:
current_date = pd.to_datetime("2022-04-01")
print(color.BOLD +  f"Current Date := {current_date}" + color.END)
print(color.BOLD +  f"===================================" + color.END)

Current Date := 2022-04-01 00:00:00


Loading the pricing data from April 

In [4]:
bond_pricing = pd.read_parquet(
    "/Users/pauladroghoff/Documents/UCL/COMP0047-Data Science/Coursework/COMP0047 Data/20220401-corporate_and_sovereign_bonds_pricing_data.parquet"
)

Filtering for good liquidity

In [5]:
bond_pricing = bond_pricing.dropna(subset=["Liquidity_Score"])
bond_pricing = bond_pricing[bond_pricing["Liquidity_Score"] != 5.0]

Ensuring to only consider bonds with matzrity after the set data to only have active bonds 

And making the maturity column a day to maturity column

In [6]:
# note: out of bounds timestamp will be NaN
bond_pricing["Maturity"] = pd.to_datetime(bond_pricing["Maturity"], errors="coerce")
bond_pricing = bond_pricing.dropna(subset=["Maturity"])
bond_pricing = bond_pricing[bond_pricing["Maturity"] > current_date]

# maturity --> #day to maturity
bond_pricing["Maturity"] = (bond_pricing["Maturity"] - current_date).dt.days

Focusing on specific columns and deleting other columns 

In [7]:
# Meta columns (identifiers)
col_meta_names = ["ISIN", "CUSIP", "Ticker", "Short_name_of_Issuer", "Currency"]

# Numerical columns (pricing, yield, duration, and spreads)
col_num_names = [
    "Bid_Price", "Mid_Price", "Ask_Price", "Bid_YTM", "Mid_YTM", "Ask_YTM",
    "Bid_Ask_Price_Spread", "Dirty_Bid_Price", "Dirty_Mid_Price", "Dirty_Ask_Price",
    "Bid_Ask_Yield_Spread", "Bid_Macaulay_Duration", "Mid_Macaulay_Duration", "Bid_Modified_Duration",
    "Bid_Convexity", "Mid_Convexity", "Ask_Convexity", "Bid_Z_Spread", "Mid_Z_Spread", "Ask_Z_Spread",
    "Spread_vs_Benchmark_Bid", "Spread_vs_Benchmark_Mid", "Spread_vs_Benchmark_Ask"
]

# Categorical columns (bond features)
col_cat_names = ["Tier", "Coupon_Type", "Defaulted", "Perpetual", "isCallable"]

# Time-related columns (dates and maturity)
col_time_names = ["Maturity", "Liquidity_asof", "Trace_Last_Trade_Date"]

# Combine all columns into one list
col_names = [*col_meta_names, *col_num_names, *col_cat_names, *col_time_names]

# Filter the DataFrame to keep only the selected columns
bond_pricing = bond_pricing[col_names]

# Optional: Verify that the columns are correctly filtered
print(bond_pricing.columns)


Index(['ISIN', 'CUSIP', 'Ticker', 'Short_name_of_Issuer', 'Currency',
       'Bid_Price', 'Mid_Price', 'Ask_Price', 'Bid_YTM', 'Mid_YTM', 'Ask_YTM',
       'Bid_Ask_Price_Spread', 'Dirty_Bid_Price', 'Dirty_Mid_Price',
       'Dirty_Ask_Price', 'Bid_Ask_Yield_Spread', 'Bid_Macaulay_Duration',
       'Mid_Macaulay_Duration', 'Bid_Modified_Duration', 'Bid_Convexity',
       'Mid_Convexity', 'Ask_Convexity', 'Bid_Z_Spread', 'Mid_Z_Spread',
       'Ask_Z_Spread', 'Spread_vs_Benchmark_Bid', 'Spread_vs_Benchmark_Mid',
       'Spread_vs_Benchmark_Ask', 'Tier', 'Coupon_Type', 'Defaulted',
       'Perpetual', 'isCallable', 'Maturity', 'Liquidity_asof',
       'Trace_Last_Trade_Date'],
      dtype='object')


Rows with NaNs

In [8]:
# Count the number of rows with NaN values in any column
rows_with_nans = bond_pricing.isna().sum(axis=1) > 0

# Show how many rows have NaNs
num_rows_with_nans = rows_with_nans.sum()
print(f"Number of rows with NaN values: {num_rows_with_nans}")

# Show how many rows would remain after dropping rows with NaNs
num_rows_remaining = bond_pricing.shape[0] - num_rows_with_nans
print(f"Number of rows remaining after dropping NaNs: {num_rows_remaining}")

# Optional: You can also see the percentage of rows with NaNs
percent_rows_with_nans = (num_rows_with_nans.sum() / bond_pricing.shape[0]) * 100
print(f"Percentage of rows with NaN values: {percent_rows_with_nans:.2f}%")

Number of rows with NaN values: 78007
Number of rows remaining after dropping NaNs: 15737
Percentage of rows with NaN values: 83.21%


Checking which columns carry a lot of NaNs

In [9]:
# Check the number of NaNs per column and sort them in descending order
nan_per_column = bond_pricing.isna().sum().sort_values(ascending=False)

# Display columns with NaNs, you can set a threshold to show only columns with significant NaNs (e.g., > 10%)
nan_per_column[nan_per_column > 0]


Trace_Last_Trade_Date      69323
Ask_Z_Spread               39588
Bid_Z_Spread               39584
Mid_Z_Spread               20174
Ask_Convexity              19536
Mid_Convexity              19536
Mid_Macaulay_Duration      19536
Bid_Macaulay_Duration      19528
Bid_Convexity              19528
Bid_Modified_Duration      19528
Spread_vs_Benchmark_Ask    11335
Spread_vs_Benchmark_Mid    11335
Spread_vs_Benchmark_Bid    11329
CUSIP                       7874
Bid_Ask_Yield_Spread        6753
Ask_YTM                     6751
Mid_YTM                     6747
Bid_YTM                     6743
Dirty_Mid_Price             2440
Dirty_Ask_Price             2440
Dirty_Bid_Price             2433
Tier                          75
Coupon_Type                   18
Bid_Ask_Price_Spread           7
Ask_Price                      7
Mid_Price                      7
dtype: int64

Removing columns with over 30% missing data

In [10]:
# Define a threshold for NaNs (e.g., drop columns with more than 30% missing data)
threshold = 0.3  # 30% of missing data
columns_to_drop = nan_per_column[nan_per_column > len(bond_pricing) * threshold].index.tolist()

# Drop the columns with high NaNs
bond_pricing = bond_pricing.drop(columns=columns_to_drop)

# Check remaining columns after dropping
print(f"Columns removed: {columns_to_drop}")
print(f"Remaining columns: {bond_pricing.columns}")


Columns removed: ['Trace_Last_Trade_Date', 'Ask_Z_Spread', 'Bid_Z_Spread']
Remaining columns: Index(['ISIN', 'CUSIP', 'Ticker', 'Short_name_of_Issuer', 'Currency',
       'Bid_Price', 'Mid_Price', 'Ask_Price', 'Bid_YTM', 'Mid_YTM', 'Ask_YTM',
       'Bid_Ask_Price_Spread', 'Dirty_Bid_Price', 'Dirty_Mid_Price',
       'Dirty_Ask_Price', 'Bid_Ask_Yield_Spread', 'Bid_Macaulay_Duration',
       'Mid_Macaulay_Duration', 'Bid_Modified_Duration', 'Bid_Convexity',
       'Mid_Convexity', 'Ask_Convexity', 'Mid_Z_Spread',
       'Spread_vs_Benchmark_Bid', 'Spread_vs_Benchmark_Mid',
       'Spread_vs_Benchmark_Ask', 'Tier', 'Coupon_Type', 'Defaulted',
       'Perpetual', 'isCallable', 'Maturity', 'Liquidity_asof'],
      dtype='object')


Checking again the imapct of removing rows with NaN content

In [11]:
# Count the number of rows with NaN values in any column
rows_with_nans = bond_pricing.isna().sum(axis=1) > 0

# Show how many rows have NaNs
num_rows_with_nans = rows_with_nans.sum()
print(f"Number of rows with NaN values: {num_rows_with_nans}")

# Show how many rows would remain after dropping rows with NaNs
num_rows_remaining = bond_pricing.shape[0] - num_rows_with_nans
print(f"Number of rows remaining after dropping NaNs: {num_rows_remaining}")

# Optional: You can also see the percentage of rows with NaNs
percent_rows_with_nans = (num_rows_with_nans.sum() / bond_pricing.shape[0]) * 100
print(f"Percentage of rows with NaN values: {percent_rows_with_nans:.2f}%")

Number of rows with NaN values: 30201
Number of rows remaining after dropping NaNs: 63543
Percentage of rows with NaN values: 32.22%


Removing the rows with NaNs

In [12]:
# Drop rows with NaNs
bond_pricing = bond_pricing.dropna().reset_index(drop=True)

# Check how many rows are left after dropping NaNs
print(f"Number of rows after dropping NaNs: {len(bond_pricing)}")

# Check the first few rows to verify the changes
print(bond_pricing.head())

Number of rows after dropping NaNs: 63543
           ISIN      CUSIP  Ticker Short_name_of_Issuer Currency  Bid_Price  \
0  NL0000003580  N4580ZKD8  NETHRS            Kdom Neth      EUR   86.91900   
1  NL0000003531  N4580ZKF3  NETHRS            Kdom Neth      EUR   93.02500   
2  NL0000003572  N4580ZKH9  NETHRS            Kdom Neth      EUR   88.11700   
3  NL0000003523  N81728AH5  NETHRS            Kdom Neth      EUR   94.84992   
4  NL0000003564  N4580ZKJ5  NETHRS            Kdom Neth      EUR   89.23200   

   Mid_Price  Ask_Price  Bid_YTM  Mid_YTM  ...  Spread_vs_Benchmark_Bid  \
0   86.99400   87.06900  1.02266  1.01634  ...                 48.46494   
1   93.10000   93.17500  0.82681  0.81755  ...                 34.95224   
2   88.19200   88.26700  0.99472  0.98799  ...                 45.67049   
3   94.85492   94.85992  0.68186  0.68118  ...                 25.99862   
4   89.30700   89.38200  0.97178  0.96457  ...                 49.44915   

   Spread_vs_Benchmark_Mid  Spre

Viewing data

In [13]:
bond_pricing

,ISIN,CUSIP,Ticker,Short_name_of_Issuer,Currency,Bid_Price,Mid_Price,Ask_Price,Bid_YTM,Mid_YTM,...,Spread_vs_Benchmark_Bid,Spread_vs_Benchmark_Mid,Spread_vs_Benchmark_Ask,Tier,Coupon_Type,Defaulted,Perpetual,isCallable,Maturity,Liquidity_asof
0,NL0000003580,N4580ZKD8,NETHRS,Kdom Neth,EUR,86.91900,86.99400,87.06900,1.02266,1.01634,...,48.46494,48.09788,47.73142,SNRFOR,Fixed,False,False,False,5037,2022-03-31
1,NL0000003531,N4580ZKF3,NETHRS,Kdom Neth,EUR,93.02500,93.10000,93.17500,0.82681,0.81755,...,34.95224,34.55710,34.16279,SNRFOR,Fixed,False,False,False,3211,2022-03-31
2,NL0000003572,N4580ZKH9,NETHRS,Kdom Neth,EUR,88.11700,88.19200,88.26700,0.99472,0.98799,...,45.67049,45.26352,44.85716,SNRFOR,Fixed,False,False,False,4672,2022-03-31
3,NL0000003523,N81728AH5,NETHRS,Kdom Neth,EUR,94.84992,94.85492,94.85992,0.68186,0.68118,...,25.99862,26.24484,26.49107,SNRFOR,Fixed,False,False,False,2846,2022-03-31
4,NL0000003564,N4580ZKJ5,NETHRS,Kdom Neth,EUR,89.23200,89.30700,89.38200,0.97178,0.96457,...,49.44915,49.25931,49.07012,SNRFOR,Fixed,False,False,False,4307,2022-03-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63538,DE000WGZ7VT6,D966BZEL8,DZBK,DZ Bk AG,EUR,102.07868,102.13562,102.19261,0.60013,0.57588,...,62.58774,61.08774,59.58774,SNRLAC,Fixed,False,False,False,857,2022-03-31
63539,DE000WGZ8FT7,D966CKBL3,DZBK,DZ Bk AG,EUR,99.75000,99.93250,100.11500,1.27845,1.22084,...,115.62189,110.55531,105.50259,SNRLAC,Fixed,False,False,False,1193,2022-03-31
63540,DE000WGZ8NG8,D966CKRG7,DZBK,DZ Bk AG,EUR,100.16022,100.23530,100.31046,0.85515,0.83428,...,64.94519,63.44519,61.94519,SNRLAC,Fixed,False,False,False,1342,2022-03-31
63541,DE000WGZ8P49,D966CKWV8,DZBK,DZ Bk AG,EUR,100.48052,100.55795,100.63545,0.86977,0.84890,...,66.40733,64.90733,63.40733,SNRLAC,Fixed,False,False,False,1382,2022-03-31


Loading the rating data

In [14]:
bond_rating = pd.read_parquet("/Users/pauladroghoff/Documents/UCL/COMP0047-Data Science/Coursework/COMP0047 Data/20220401-Instrument_Rating_reference_data.parquet")

bond_rating["instrumentRating.effectiveDate"] = pd.to_datetime(bond_rating["instrumentRating.effectiveDate"]) 

bond_rating

,id,isin,cusip,instrumentRating.directionOfChange,instrumentRating.effectiveDate,instrumentRating.endDate,instrumentRating.outlook,instrumentRating.provider,instrumentRating.rating,instrumentRating.type,instrumentRating.watch
0,9,JP356680BBC5,J05523CL7,Downgrade,2021-06-25,None,None,MOODYS,A2,Long-Term Debt Rating Senior Unsecured LC,None
1,1336200,US12489WMR24,12489WMR2,DECISION NOT TO RATE,2005-08-29,None,None,MOODYS,NR,Long-Term Debt Rating Senior Subordinate LC,None
2,1336200,US12489WMR24,12489WMR2,Withdrawn,2014-06-25,None,None,FITCH,WD,Long-Term Issue Rating,None
3,1336201,US63937HAG39,63937HAG3,None,2016-11-17,None,None,SNP,NR,Long-Term Local Currency,None
4,1336202,US12667G5S86,12667G5S8,None,2018-02-06,None,None,SNP,NR,Long-Term Local Currency,None
...,...,...,...,...,...,...,...,...,...,...,...
4107762,33240396,US271014LK61,271014LK6,None,2001-05-31,None,None,SNP,AAA,Long-Term Rating,None
4107763,33240395,US271014LH33,271014LH3,None,2001-05-31,None,None,MOODYS,Aaa,Long-Term Rating,None
4107764,33240395,US271014LH33,271014LH3,None,2001-05-31,None,None,SNP,AAA,Long-Term Rating,None
4107765,33240394,US271014LJ98,271014LJ9,None,2001-05-31,None,None,MOODYS,Aaa,Long-Term Rating,None


Check the NaNs per column

In [15]:
# Check the number of NaNs per provider
na_counts = bond_rating.isna().sum(axis=0)
print("NaN counts per column:")
print(na_counts)

NaN counts per column:
id                                          0
isin                                     6428
cusip                                  170443
instrumentRating.directionOfChange     886585
instrumentRating.effectiveDate           3273
instrumentRating.endDate              4107767
instrumentRating.outlook              2360835
instrumentRating.provider                   0
instrumentRating.rating                     0
instrumentRating.type                       0
instrumentRating.watch                4040405
dtype: int64


Drop rows with no ISIN and delete ISIN duplicates (for ISING rows with all columns the same)

In [16]:
# Only use rows where ISIN is not null
bond_rating = bond_rating[bond_rating["isin"].notna()]

# Drop duplicates and reset index
bond_rating = bond_rating.drop_duplicates().reset_index(drop=True)

# Print the first 3 rows to check the data
print(bond_rating.head(3))

        id          isin      cusip instrumentRating.directionOfChange  \
0        9  JP356680BBC5  J05523CL7                          Downgrade   
1  1336200  US12489WMR24  12489WMR2               DECISION NOT TO RATE   
2  1336200  US12489WMR24  12489WMR2                          Withdrawn   

  instrumentRating.effectiveDate instrumentRating.endDate  \
0                     2021-06-25                     None   
1                     2005-08-29                     None   
2                     2014-06-25                     None   

  instrumentRating.outlook instrumentRating.provider instrumentRating.rating  \
0                     None                    MOODYS                      A2   
1                     None                    MOODYS                      NR   
2                     None                     FITCH                      WD   

                         instrumentRating.type instrumentRating.watch  
0    Long-Term Debt Rating Senior Unsecured LC                   

Number of rows in the dataset

In [17]:
# Count the number of rows in the bond_rating dataset
print(f"Instrument rating number := {bond_rating.shape[0]}")

Instrument rating number := 4101295


endDate values 

In [18]:
# Check for any endDate values (they seem to be mostly NaN in your case)
print("\nInstrumentRating.endDate value counts:")
print(bond_rating["instrumentRating.endDate"].value_counts(dropna=False))


InstrumentRating.endDate value counts:
instrumentRating.endDate
None    4101295
Name: count, dtype: int64


Rating providers and rating values 

In [19]:
# Check unique providers and ratings
print("\nUnique rating providers:")
print(bond_rating['instrumentRating.provider'].unique())
print("\nUnique ratings:")
print(bond_rating['instrumentRating.rating'].unique())


Unique rating providers:
['MOODYS' 'FITCH' 'SNP' 'KROLL' 'MARC' 'RAM']

Unique ratings:
['A2' 'NR' 'WD' 'WR' 'PIF' 'Baa2' 'CCC (sf)' 'AA+ (sf)' 'Aa1' 'A1' 'BBB+p'
 'CCC' 'A+ (sf)' 'CC (sf)' 'AA+p (sf)' 'BBB+/A-2' 'NR/NR' 'AAA' 'Aaa' 'AA'
 'Baa3' 'A (sf)' 'BBB- (sf)' 'BBB' 'BBB+ (sf)' 'A' 'D (sf)' 'BB (sf)' 'Ca'
 'D' 'AA (sf)' 'P-2(Low)' 'BBB-' 'AA+' 'A-' 'Caa3' 'B- (sf)' 'Baa1' 'Ba1'
 'BBB (sf)' 'C' 'Ba2' 'BB+' 'AA+/A-1+' 'Aa2' 'AAA (sf)' 'BB- (sf)'
 'A-/A-2' 'Aa3' 'A (idn)' 'AAA (idn)' 'A3' 'AA-' 'A+' 'A+/A-1+' 'BBB+'
 '85 - LGD5' 'B3' 'B+' '6(0%)' 'B1' 'VMIG 1' 'B2' '1+' '45 - LGD3'
 '2(75%)' 'RR2' 'BB-' '71 - LGD5' 'B' '5(25%)' '81 - LGD5' 'BB' '4(30%)'
 '84 - LGD5' 'Caa1' 'B-' '5(10%)' 'B+ (sf)' 'RR4' '73 - LGD5' 'Ba3'
 '30 - LGD3' '1(95%)' 'RR1' 'A/A-1' 'CC' '40 - LGD3' '3(55%)' '77 - LGD5'
 '5(20%)' 'AA-/A-1+' '3(65%)' '92 - LGD6' 'CCC+' '58 - LGD4' 'Caa2' 'A-p'
 'BB+ (sf)' 'AA+p' 'A- (sf)' 'AA- (sf)' 'CCC- (sf)' 'A+/A-1' '93 - LGD6'
 'AA (mex)' '50 - LGD4' 'RR6' '53 - LGD4' '2(

Number of bonds per rating

In [20]:
# Check how many times each rating appears
pd.options.display.max_rows = 100
print("\nRating counts:")
print(bond_rating["instrumentRating.rating"].value_counts())


Rating counts:
instrumentRating.rating
NR            1137304
WR             500734
AA             377641
AAA            237097
AA+            227990
               ...   
AAA (pan)           1
twA (sf)            1
twAAA (sf)          1
(P)Baa2             1
SG                  1
Name: count, Length: 585, dtype: int64


In [21]:
# Check the value counts for ISIN after filtering
print("\nValue counts for ISIN:")
print(bond_rating["isin"].value_counts(dropna=False))


Value counts for ISIN:
isin
US882721BN27    11
US882721BM44    11
US13063A6J80    10
US45471ADV17    10
US786134EQ58    10
                ..
US491207VV59     1
CH0118628251     1
IT0004883374     1
IT0004883051     1
JP356680BBC5     1
Name: count, Length: 2159982, dtype: int64


Deleting duplicates in ISINs with multiple ratings from the same rating provider 

In [22]:
# Sort by ISIN, provider, and effective date (descending to get most recent first)
bond_rating = bond_rating.sort_values(
    ["isin", "instrumentRating.provider", "instrumentRating.effectiveDate"], 
    ascending=[True, True, False]  # Ascending by ISIN and provider, but descending by date
)

# Drop duplicates, keeping the first (most recent) for each ISIN and rating provider combination
bond_rating = bond_rating.drop_duplicates(subset=["isin", "instrumentRating.provider"], keep="first")

# Show the cleaned-up data
bond_rating.head()

,id,isin,cusip,instrumentRating.directionOfChange,instrumentRating.effectiveDate,instrumentRating.endDate,instrumentRating.outlook,instrumentRating.provider,instrumentRating.rating,instrumentRating.type,instrumentRating.watch
4059800,28436861,AMACBAB21ER0,None,Matured,2020-02-18,None,None,FITCH,NR,Long-Term Issue Rating,None
3042002,30994508,AMACBAB23ER6,Y000W9AA6,Matured,2021-07-12,None,None,FITCH,NR,Long-Term Issue Rating,None
1679077,26394033,AMAMRBB23ER9,None,None,2017-08-23,None,None,FITCH,WD,Long-Term Issue Rating,None
3618979,28554902,AMGB1029A276,Y5S02PJA3,Affirmed,2022-03-18,None,None,FITCH,B+,Long-Term Issue Rating,None
2586732,29206030,AMGT52042190,None,None,2019-02-04,None,None,FITCH,NR,Short-Term Issue Rating,None


Focusing on one of the rating providers for better dataset merging

In [23]:
print(bond_rating.columns)

# Count the number of ISINs for each rating provider
isin_counts_by_provider = bond_rating.groupby("instrumentRating.provider")["isin"].nunique()

# Print the counts to see which provider has the most ISINs rated
print(isin_counts_by_provider)

Index(['id', 'isin', 'cusip', 'instrumentRating.directionOfChange',
       'instrumentRating.effectiveDate', 'instrumentRating.endDate',
       'instrumentRating.outlook', 'instrumentRating.provider',
       'instrumentRating.rating', 'instrumentRating.type',
       'instrumentRating.watch'],
      dtype='object')
instrumentRating.provider
FITCH      547978
KROLL       90861
MARC          957
MOODYS    1134403
RAM          2714
SNP       1451300
Name: isin, dtype: int64


Filter for SNP as this provider covers most ISINs

In [24]:
bond_rating = bond_rating[bond_rating["instrumentRating.provider"] == "SNP"]

Merge the pricing April data and the SNP rating

In [25]:
print(bond_pricing.columns)
# Renaming the isin column to be the same across datasets
bond_rating = bond_rating.rename(columns={"isin": "ISIN"})
# Merge the bond pricing data with the bond rating data on the 'ISIN' column
bond_pricing = bond_pricing.merge(bond_rating, on="ISIN", how="left")
print(bond_pricing.columns)

Index(['ISIN', 'CUSIP', 'Ticker', 'Short_name_of_Issuer', 'Currency',
       'Bid_Price', 'Mid_Price', 'Ask_Price', 'Bid_YTM', 'Mid_YTM', 'Ask_YTM',
       'Bid_Ask_Price_Spread', 'Dirty_Bid_Price', 'Dirty_Mid_Price',
       'Dirty_Ask_Price', 'Bid_Ask_Yield_Spread', 'Bid_Macaulay_Duration',
       'Mid_Macaulay_Duration', 'Bid_Modified_Duration', 'Bid_Convexity',
       'Mid_Convexity', 'Ask_Convexity', 'Mid_Z_Spread',
       'Spread_vs_Benchmark_Bid', 'Spread_vs_Benchmark_Mid',
       'Spread_vs_Benchmark_Ask', 'Tier', 'Coupon_Type', 'Defaulted',
       'Perpetual', 'isCallable', 'Maturity', 'Liquidity_asof'],
      dtype='object')
Index(['ISIN', 'CUSIP', 'Ticker', 'Short_name_of_Issuer', 'Currency',
       'Bid_Price', 'Mid_Price', 'Ask_Price', 'Bid_YTM', 'Mid_YTM', 'Ask_YTM',
       'Bid_Ask_Price_Spread', 'Dirty_Bid_Price', 'Dirty_Mid_Price',
       'Dirty_Ask_Price', 'Bid_Ask_Yield_Spread', 'Bid_Macaulay_Duration',
       'Mid_Macaulay_Duration', 'Bid_Modified_Duration', 'Bid_Con

Dropping empty rows after merge, ISINs tha are not covered by the SNP rating 

In [26]:
# Number of rows after merging but before dropping NaNs
rows_after_merge = bond_pricing.shape[0]

# Drop rows where the rating is NaN
bond_pricing = bond_pricing.dropna(subset=["instrumentRating.rating"])

# Number of rows after dropping NaNs
rows_after_drop = bond_pricing.shape[0]

# Calculate the difference
rows_dropped = rows_after_merge - rows_after_drop

# Print the results
print(f"Rows after merging: {rows_after_merge}")
print(f"Rows after dropping rows with no rating: {rows_after_drop}")
print(f"Rows dropped due to missing ratings: {rows_dropped}")
print(f"Percentage of rows dropped: {rows_dropped / rows_after_merge * 100:.2f}%")

# Optionally, check the value counts of the ratings column
print(bond_pricing["instrumentRating.rating"].value_counts())

Rows after merging: 63543
Rows after dropping rows with no rating: 29732
Rows dropped due to missing ratings: 33811
Percentage of rows dropped: 53.21%
instrumentRating.rating
BBB+          3194
A+            3143
AA+           2705
BBB           2624
A-            2331
AAA           2136
BBB-          1900
A             1867
BB+           1235
AA-           1223
BB-           1155
AA            1002
BB             922
B+             892
B              829
B-             618
NR             517
CCC+           437
CCC            227
CCC-           168
3(65%)          54
AA (sf)         36
A+ (sf)         29
4(45%)          26
BBB (sf)        25
A (sf)          25
3(50%)          24
mxAAA           20
BBB- (sf)       20
AA- (sf)        19
D               18
4(35%)          18
BBB+ (sf)       17
A- (sf)         17
BB (sf)         16
1+              16
4(30%)          16
2(75%)          16
4(40%)          15
CC              14
BBB prelim      12
B (sf)          12
AAA (sf)        11
NR preli

Include March pricing data now to calculate returns instead of prices 

In [27]:
print(color.BOLD + f"Current Date is set to := {current_date}")

Current Date is set to := 2022-04-01 00:00:00


In [28]:
# previous_date = current_date - pd.Timedelta("1D")
previous_date = pd.to_datetime("2022-01-03")
# load data 
previous_bond_pricing = pd.read_parquet("/Users/pauladroghoff/Documents/UCL/COMP0047-Data Science/Coursework/COMP0047 Data/20220103-corporate_and_sovereign_bonds_pricing_data.parquet")

In [29]:
# Only look at the ISIN and the mid price columns and drop the rows for which these column entries are empty 
# Also rename the mid price column 
previous_bond_pricing = previous_bond_pricing[
    ["ISIN", "Mid_Price"]
].dropna().rename(columns={"Mid_Price": "Previous_Mid_Price"})

Merge the previous mid price from the march with the big bond pricing dataset (from April) including the ratings

In [30]:
# Merging 
bond_pricing = bond_pricing.merge(
    previous_bond_pricing, on="ISIN", how="left"
)

Calculating the returns 

In [31]:
# First checking if the previous date is set correctly 
print(color.BOLD + f"Previous Date is set to := {previous_date}")
print(color.BOLD + f"Current Date is set to := {current_date}")

Previous Date is set to := 2022-01-03 00:00:00
Current Date is set to := 2022-04-01 00:00:00


In [32]:
# Calculating the return as follows: (today's (April) mid price - previous (March) mid price)/ previous (March) mid price
bond_pricing["Return"] = (
    bond_pricing["Mid_Price"] - bond_pricing["Previous_Mid_Price"]
) / bond_pricing["Previous_Mid_Price"]

print(bond_pricing.columns)


Index(['ISIN', 'CUSIP', 'Ticker', 'Short_name_of_Issuer', 'Currency',
       'Bid_Price', 'Mid_Price', 'Ask_Price', 'Bid_YTM', 'Mid_YTM', 'Ask_YTM',
       'Bid_Ask_Price_Spread', 'Dirty_Bid_Price', 'Dirty_Mid_Price',
       'Dirty_Ask_Price', 'Bid_Ask_Yield_Spread', 'Bid_Macaulay_Duration',
       'Mid_Macaulay_Duration', 'Bid_Modified_Duration', 'Bid_Convexity',
       'Mid_Convexity', 'Ask_Convexity', 'Mid_Z_Spread',
       'Spread_vs_Benchmark_Bid', 'Spread_vs_Benchmark_Mid',
       'Spread_vs_Benchmark_Ask', 'Tier', 'Coupon_Type', 'Defaulted',
       'Perpetual', 'isCallable', 'Maturity', 'Liquidity_asof', 'id', 'cusip',
       'instrumentRating.directionOfChange', 'instrumentRating.effectiveDate',
       'instrumentRating.endDate', 'instrumentRating.outlook',
       'instrumentRating.provider', 'instrumentRating.rating',
       'instrumentRating.type', 'instrumentRating.watch', 'Previous_Mid_Price',
       'Return'],
      dtype='object')


In [33]:
# Dropping the previous mid price column as it is not needed anymore 
# bond_pricing = bond_pricing.drop(columns=["Previous_Mid_Price"])

print(col_num_names)
# Add return column to the other numerical columns 
col_num_names.append("Return")  # Add "Return" to numerical columns list
print(col_num_names)

print(bond_pricing.columns)

['Bid_Price', 'Mid_Price', 'Ask_Price', 'Bid_YTM', 'Mid_YTM', 'Ask_YTM', 'Bid_Ask_Price_Spread', 'Dirty_Bid_Price', 'Dirty_Mid_Price', 'Dirty_Ask_Price', 'Bid_Ask_Yield_Spread', 'Bid_Macaulay_Duration', 'Mid_Macaulay_Duration', 'Bid_Modified_Duration', 'Bid_Convexity', 'Mid_Convexity', 'Ask_Convexity', 'Bid_Z_Spread', 'Mid_Z_Spread', 'Ask_Z_Spread', 'Spread_vs_Benchmark_Bid', 'Spread_vs_Benchmark_Mid', 'Spread_vs_Benchmark_Ask']
['Bid_Price', 'Mid_Price', 'Ask_Price', 'Bid_YTM', 'Mid_YTM', 'Ask_YTM', 'Bid_Ask_Price_Spread', 'Dirty_Bid_Price', 'Dirty_Mid_Price', 'Dirty_Ask_Price', 'Bid_Ask_Yield_Spread', 'Bid_Macaulay_Duration', 'Mid_Macaulay_Duration', 'Bid_Modified_Duration', 'Bid_Convexity', 'Mid_Convexity', 'Ask_Convexity', 'Bid_Z_Spread', 'Mid_Z_Spread', 'Ask_Z_Spread', 'Spread_vs_Benchmark_Bid', 'Spread_vs_Benchmark_Mid', 'Spread_vs_Benchmark_Ask', 'Return']
Index(['ISIN', 'CUSIP', 'Ticker', 'Short_name_of_Issuer', 'Currency',
       'Bid_Price', 'Mid_Price', 'Ask_Price', 'Bid_YTM

In [34]:
# Print final shape of the bond pricing data
print(bond_pricing.shape)

(29732, 45)


Saving the completely merged dataset(April data, return column, rating)

In [35]:
bond_pricing.to_parquet("Bond_Pricing_With_Returns_And_Ratings.parquet")

Exploring the Prganisations 

In [36]:
bond_rating_org = pd.read_parquet(
    "/Users/pauladroghoff/Documents/UCL/COMP0047-Data Science/Coursework/COMP0047 Data/20220401-Organisation_Rating.parquet"
)
print(bond_rating_org.columns)
print(bond_pricing.columns)

Index(['id', 'organizationId', 'ticker', 'legacyparentTicker',
       'legacyTopParentTicker', 'parentTicker', 'topParentTicker',
       'organizationRating.directionOfChange',
       'organizationRating.effectiveDate', 'organizationRating.endDate',
       'organizationRating.outlook', 'organizationRating.provider',
       'organizationRating.rating', 'organizationRating.type',
       'organizationRating.watch'],
      dtype='object')
Index(['ISIN', 'CUSIP', 'Ticker', 'Short_name_of_Issuer', 'Currency',
       'Bid_Price', 'Mid_Price', 'Ask_Price', 'Bid_YTM', 'Mid_YTM', 'Ask_YTM',
       'Bid_Ask_Price_Spread', 'Dirty_Bid_Price', 'Dirty_Mid_Price',
       'Dirty_Ask_Price', 'Bid_Ask_Yield_Spread', 'Bid_Macaulay_Duration',
       'Mid_Macaulay_Duration', 'Bid_Modified_Duration', 'Bid_Convexity',
       'Mid_Convexity', 'Ask_Convexity', 'Mid_Z_Spread',
       'Spread_vs_Benchmark_Bid', 'Spread_vs_Benchmark_Mid',
       'Spread_vs_Benchmark_Ask', 'Tier', 'Coupon_Type', 'Defaulted',
      

In [37]:
# Number of tickers ine ach dataset
num_bond_tickers = bond_pricing["Ticker"].nunique()
num_org_tickers = bond_rating_org["ticker"].nunique()

print(f"Unique tickers in bond dataset: {num_bond_tickers}")
print(f"Unique tickers in organization dataset: {num_org_tickers}")

Unique tickers in bond dataset: 4929
Unique tickers in organization dataset: 366027


In [38]:
# Find overlapping tickers 
# Convert tickers to sets (drop NaNs to avoid issues)
bond_tickers = set(bond_pricing["Ticker"].dropna().unique())
org_tickers = set(bond_rating_org["ticker"].dropna().unique())

# Find common tickers
common_tickers = bond_tickers.intersection(org_tickers)
num_common_tickers = len(common_tickers)

print(f"Tickers in both datasets: {num_common_tickers}")

coverage_percentage = (num_common_tickers / num_bond_tickers) * 100
print(f"Percentage of bond tickers covered by organization ratings: {coverage_percentage:.2f}%")


Tickers in both datasets: 4929
Percentage of bond tickers covered by organization ratings: 100.00%


This implies that all the tickers in the bond_pricing data are covered in the org ratings 

Integrating the org ticker data into the bond_pricing data

In [39]:
# Step 1: Keep only the latest organization rating for each ticker
bond_rating_org = bond_rating_org.sort_values(["ticker", "organizationRating.effectiveDate"], ascending=False)
bond_rating_org = bond_rating_org.drop_duplicates(subset=["ticker"], keep="first")

# Step 2: Merge with bond_pricing using "Ticker"
bond_pricing = bond_pricing.merge(
    bond_rating_org[["ticker", "organizationRating.provider", "organizationRating.rating", 
                     "organizationRating.effectiveDate", "organizationRating.outlook"]],
    left_on="Ticker", 
    right_on="ticker", 
    how="left"
)

# Step 3: Drop the duplicate ticker column from organization ratings
bond_pricing = bond_pricing.drop(columns=["ticker"])

# Step 4: Rename columns for clarity
bond_pricing = bond_pricing.rename(columns={
    "organizationRating.provider": "Org_Rating_Provider",
    "organizationRating.rating": "Org_Rating",
    "organizationRating.effectiveDate": "Org_Rating_EffectiveDate",
    "organizationRating.outlook": "Org_Outlook"
})

# Step 5: Check the merge results
print(bond_pricing[["Ticker", "Org_Rating_Provider", "Org_Rating"]].head())


      Ticker Org_Rating_Provider Org_Rating
0        KFW               FITCH         WD
1    FANTHOL                 SNP         NR
2        EIB               FITCH        AAA
3      EURSA              MOODYS         WR
4  KAZAKS-Bk               FITCH         WD


Look at org rating providers and bond rating providers 

In [40]:
print(bond_pricing.head())

           ISIN      CUSIP     Ticker         Short_name_of_Issuer Currency  \
0  AU0000002073  D4S475HJ3        KFW       Kred Fuer Wiederaufbau      AUD   
1  XS2100005771  G3311PAH8    FANTHOL  Fantasia Hldgs Group Co Ltd      USD   
2  XS2100001192  B3821TPQ2        EIB                Eurpn Invt Bk      NOK   
3  KZ2C00003051  Y2R48JAH5      EURSA              Eurasian Bk JSC      KZT   
4  KZ2C00003002  Y4S4C0AA1  KAZAKS-Bk         Dev Bk of Kazakhstan      KZT   

   Bid_Price  Mid_Price  Ask_Price    Bid_YTM    Mid_YTM  ...  \
0  101.27922  101.32972  101.38022    1.39587    1.34106  ...   
1   14.12656   14.45781   14.78906  556.43932  545.58401  ...   
2   97.79000   97.81500   97.84000    2.53994    2.53086  ...   
3   85.50855   85.64425   85.78021   15.38859   15.32251  ...   
4   86.17503   86.29733   86.41985   14.41116   14.35008  ...   

   instrumentRating.provider  instrumentRating.rating  \
0                        SNP                      AAA   
1                   

In [41]:
print(bond_pricing.columns)

Index(['ISIN', 'CUSIP', 'Ticker', 'Short_name_of_Issuer', 'Currency',
       'Bid_Price', 'Mid_Price', 'Ask_Price', 'Bid_YTM', 'Mid_YTM', 'Ask_YTM',
       'Bid_Ask_Price_Spread', 'Dirty_Bid_Price', 'Dirty_Mid_Price',
       'Dirty_Ask_Price', 'Bid_Ask_Yield_Spread', 'Bid_Macaulay_Duration',
       'Mid_Macaulay_Duration', 'Bid_Modified_Duration', 'Bid_Convexity',
       'Mid_Convexity', 'Ask_Convexity', 'Mid_Z_Spread',
       'Spread_vs_Benchmark_Bid', 'Spread_vs_Benchmark_Mid',
       'Spread_vs_Benchmark_Ask', 'Tier', 'Coupon_Type', 'Defaulted',
       'Perpetual', 'isCallable', 'Maturity', 'Liquidity_asof', 'id', 'cusip',
       'instrumentRating.directionOfChange', 'instrumentRating.effectiveDate',
       'instrumentRating.endDate', 'instrumentRating.outlook',
       'instrumentRating.provider', 'instrumentRating.rating',
       'instrumentRating.type', 'instrumentRating.watch', 'Previous_Mid_Price',
       'Return', 'Org_Rating_Provider', 'Org_Rating',
       'Org_Rating_Effectiv

In [42]:
# List of columns you want to print
columns_to_print = [
    'ISIN', 
    'Ticker', 
    'Org_Rating_Provider', 
    'Org_Rating', 
    'instrumentRating.provider', 
    'instrumentRating.rating'
]

# Print the selected columns
print(bond_pricing[columns_to_print].head())  # Display the first few rows of these columns


           ISIN     Ticker Org_Rating_Provider Org_Rating  \
0  AU0000002073        KFW               FITCH         WD   
1  XS2100005771    FANTHOL                 SNP         NR   
2  XS2100001192        EIB               FITCH        AAA   
3  KZ2C00003051      EURSA              MOODYS         WR   
4  KZ2C00003002  KAZAKS-Bk               FITCH         WD   

  instrumentRating.provider instrumentRating.rating  
0                       SNP                     AAA  
1                       SNP                      NR  
2                       SNP                     AAA  
3                       SNP                      NR  
4                       SNP                      NR  


In [43]:
# Filter rows with a specific ticker (for example 'KFW')
specific_ticker = 'KFW'
filtered_data = bond_pricing[bond_pricing['Ticker'] == specific_ticker]

# Specify the columns to display
columns_to_display = ['ISIN', 'Ticker', 'Org_Rating_Provider', 'Org_Rating', 'instrumentRating.provider', 'instrumentRating.rating']

# Show the filtered rows with the specific columns
filtered_data = filtered_data[columns_to_display]

# Print the filtered data
print(filtered_data)


               ISIN Ticker Org_Rating_Provider Org_Rating  \
0      AU0000002073    KFW               FITCH         WD   
740    CH0029008809    KFW               FITCH         WD   
1897   XS1716607269    KFW               FITCH         WD   
2578   XS1950905486    KFW               FITCH         WD   
2641   XS1814900806    KFW               FITCH         WD   
...             ...    ...                 ...        ...   
28588  AU000KFWHAF2    KFW               FITCH         WD   
28589  AU000KFWHAA3    KFW               FITCH         WD   
28590  AU000KFWHAE5    KFW               FITCH         WD   
29146  DE000A2YNZ16    KFW               FITCH         WD   
29256  CAD6426YAA24    KFW               FITCH         WD   

      instrumentRating.provider instrumentRating.rating  
0                           SNP                     AAA  
740                         SNP                     AAA  
1897                        SNP                     AAA  
2578                        SNP    

## How does this make sense? Trying to understand the meaning behind the organisation provider and the instrument provider and their ratings

Saving the merged dataset

In [44]:
bond_pricing.to_parquet("Bond_Pricing_With_Returns_Ratings_And_Org.parquet")